In [1]:
import models.zoo as modelzoo
from utils import common
from datasets import sized_transforms
from datasets import get_dataset

import numpy as np
import torch
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import os
import time
import copy

from argparse import Namespace

In [2]:
model = modelzoo.get_net('Classifier', 32, num_classes=43, complexity=64, channel=3)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
device = torch.device('cuda')

In [4]:
imagenet = get_dataset('ImageNet64', 32, True)

In [5]:
from victim.blackbox import Blackbox
blackbox = Blackbox.from_modeldir('/mnt/ywb/results/victim/gtsrb-blackbox')

=> loading checkpoint '/mnt/ywb/results/victim/gtsrb-blackbox/checkpoint.pth.tar'
=> loaded checkpoint (epoch 56, acc=97.81)


In [6]:
selected_indices = np.load('/mnt/ywb/results/adversary/gtsrb-imagenet-targeted-full/selected_indices.npy')

In [7]:
# train_data = common.Subset(imagenet, selected_indices[:5000], 43, '/mnt/ywb/results/adversary/gtsrb-imagenet-targeted-full/')
trainset = common.query_dataset(blackbox, imagenet, list_indices=selected_indices[:10000], batch_size=64, device=torch.device('cuda'), transform=sized_transforms[32],)

100%|██████████| 10000/10000 [00:01<00:00, 5916.39it/s]


In [8]:
test_data = get_dataset('GTSRB', 32, False)

=> done loading GTSRB (test) with 12630 examples


In [9]:
test = common.query_dataset(blackbox, test_data, batch_size=64,device=torch.device('cuda'), transform=sized_transforms[32])
test.labels = [label.argmax() for label in test.labels]

100%|██████████| 12630/12630 [00:02<00:00, 5687.08it/s]


In [10]:
dataloaders = {
    'train': torch.utils.data.DataLoader(
        trainset,
        batch_size=64,
        shuffle=True,
        num_workers=4
    ),
    'val': torch.utils.data.DataLoader(
        test_data,
        batch_size=64,
        shuffle=True,
        num_workers=4
    ),
}

In [11]:
def train_model(model, train_criterion, test_criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    if phase == 'train':
                        loss = train_criterion(outputs, labels)
                    else:
                        loss = test_criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                if phase == 'val':
                    running_corrects += torch.sum(preds == labels.data)
                else:
                    running_corrects += torch.sum(preds == labels.argmax(1))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=60, gamma=0.1)

In [13]:
def SCE(input, target):
    log_probs = torch.nn.functional.log_softmax(input, dim=1)
    return -(target * log_probs).sum() / input.shape[0]

train_criterion = SCE
criterion = nn.CrossEntropyLoss()

In [14]:
dataset_sizes = {'train': len(trainset), 'val': len(test)}

In [15]:
model = train_model(model.cuda(), train_criterion, criterion, optimizer, lr_scheduler, num_epochs=100)

Epoch 0/99
----------
train Loss: 2.9181 Acc: 0.2931
val Loss: 3.1444 Acc: 0.1931

Epoch 1/99
----------
train Loss: 2.5559 Acc: 0.3858
val Loss: 2.7818 Acc: 0.2881

Epoch 2/99
----------
train Loss: 2.3370 Acc: 0.4348
val Loss: 2.4200 Acc: 0.3763

Epoch 3/99
----------
train Loss: 2.1771 Acc: 0.4949
val Loss: 2.1282 Acc: 0.4178

Epoch 4/99
----------
train Loss: 2.0596 Acc: 0.5243
val Loss: 2.0054 Acc: 0.4292

Epoch 5/99
----------
train Loss: 1.9643 Acc: 0.5621
val Loss: 1.6749 Acc: 0.5429

Epoch 6/99
----------
train Loss: 1.8926 Acc: 0.5929
val Loss: 1.5932 Acc: 0.5484

Epoch 7/99
----------
train Loss: 1.8337 Acc: 0.6017
val Loss: 1.4313 Acc: 0.6303

Epoch 8/99
----------
train Loss: 1.7798 Acc: 0.6355
val Loss: 1.4064 Acc: 0.5990

Epoch 9/99
----------
train Loss: 1.7418 Acc: 0.6498
val Loss: 1.3317 Acc: 0.5957

Epoch 10/99
----------
train Loss: 1.6987 Acc: 0.6691
val Loss: 1.2034 Acc: 0.6559

Epoch 11/99
----------
train Loss: 1.6692 Acc: 0.6877
val Loss: 1.1119 Acc: 0.7151

Ep